In [1]:
import torch.nn as nn
import numpy as np
import torch
import yaml
from torch.autograd import Variable
from torch.utils.data import DataLoader
from txt2image_dataset import Text2ImageDataset
from utils import Utils, Logger
from PIL import Image
import os

### Dataset

In [2]:
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

config

{'birds_images_path': 'data/cvpr2016_cub/images/',
 'birds_embedding_path': 'data/cub_icml/',
 'birds_text_path': 'data/cvpr2016_cub/cvpr2016_cub/text_c10/',
 'birds_dataset_path': 'data/cvpr2016_cub/text2image/birds.hdf5',
 'val_split_path': 'data/cvpr2016_cub/valclasses.txt',
 'train_split_path': 'data/cvpr2016_cub/trainclasses.txt',
 'test_split_path': 'data/cvpr2016_cub/testclasses.txt',
 'flowers_images_path': 'data/cvpr2016_flowers/images/',
 'flowers_embedding_path': 'data/flowers_icml/',
 'flowers_text_path': 'data/cvpr2016_flowers/text_c10/',
 'flowers_dataset_path': 'data/cvpr2016_flowers/text2image/flowers.hdf5',
 'flowers_val_split_path': 'data/cvpr2016_flowers/valclasses.txt',
 'flowers_train_split_path': 'data/cvpr2016_flowers/trainclasses.txt',
 'flowers_test_split_path': 'data/cvpr2016_flowers/testclasses.txt'}

### Vanilla GAN Generator and Discriminator

In [15]:
from models import gan

In [4]:
generator = gan.generator()
discriminator = gan.discriminator()
print(generator, discriminator)

generator(
  (netG): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
) 

### Load pretrained Generator and Discriminator, if they exist

In [83]:
pre_trained_disc = None #'results/gan/checkpoints_vanilla_gan/disc_90.pth'
pre_trained_gen = None #'results/gan/checkpoints_vanilla_gan/gen_90.pth'

## initialize the generator and discriminator ##
generator = generator.cuda()
discriminator = discriminator.cuda()
################################################

def load_gen_and_disc(pre_trained_disc, pre_trained_gen):
    epochs_trained = int(pre_trained_disc.split("/")[-1].split('_')[-1].split('.')[0])
    generator.load_state_dict(torch.load(pre_trained_gen))
    discriminator.load_state_dict(torch.load(pre_trained_disc))
    return generator, discriminator, epochs_trained

epochs_trained = 0

if pre_trained_disc and pre_trained_gen:
    generator, discriminator, epochs_trained = load_gen_and_disc(pre_trained_disc, 
                                                 pre_trained_gen)
else:
    generator.apply(Utils.weights_init)
    discriminator.apply(Utils.weights_init)

### Training a vanilla GAN

In [23]:
from models import gan

In [24]:
noise_dim = 100
batch_size = 64
lr = 0.0002
beta1 = 0.5
num_epochs = 90
l1_coef = 50
l2_coef = 200

vis_screen = 'vanilla_gan_loss'
checkpoints_path = 'checkpoints_vanilla_gan'
save_path = 'results/gan'
logger = Logger(vis_screen)

Setting up a new session...


In [25]:
criterion = nn.BCELoss()
l2_loss = nn.MSELoss()
l1_loss = nn.L1Loss()

# optimG = torch.optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
# optimD = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
def train_vanilla_gan(ds='flowers'):

    if ds == 'birds':
        dataset = Text2ImageDataset(config['birds_dataset_path'], split=0)
    if ds == 'flowers':
        dataset = Text2ImageDataset(config['flowers_dataset_path'], split=0)

    data_loader = DataLoader(dataset,
                            batch_size=batch_size,
                            shuffle=True)

    iteration = 0
    print("num epochs", epochs, "num iterations", len(data_loader), 'batch_size', batch_size)
    
    for epoch in range(num_epochs+1):
        for sample in data_loader:
            iteration += 1
            right_images = sample['right_images']

            right_images = Variable(right_images.float()).cuda()

            real_labels = torch.ones(right_images.size(0))
            fake_labels = torch.zeros(right_images.size(0))

            smoothed_real_labels = torch.FloatTensor(Utils.smooth_label(real_labels.numpy(), -0.1))

            real_labels = Variable(real_labels).cuda()
            smoothed_real_labels = Variable(smoothed_real_labels).cuda()
            fake_labels = Variable(fake_labels).cuda()

            # Train the discriminator
            discriminator.zero_grad()
            outputs, activation_real = discriminator(right_images)
            # print(f"outputs shape {outputs.shape}, activation shape {activation_real.shape}")
            real_loss = criterion(outputs, smoothed_real_labels)
            real_score = outputs

            noise = Variable(torch.randn(right_images.size(0), 100)).cuda()
            noise = noise.view(noise.size(0), 100, 1, 1)
            # print(f"noise shape {noise.shape}")
            fake_images = generator(noise)
            # print(f"fake_images shape {fake_images.shape}")
            outputs, _ = discriminator(fake_images)
            fake_loss = criterion(outputs, fake_labels)
            fake_score = outputs

            d_loss = real_loss + fake_loss

            d_loss.backward()
            optimD.step()

            # Train the generator
            generator.zero_grad()

            noise = Variable(torch.randn(right_images.size(0), 100)).cuda()
            noise = noise.view(noise.size(0), 100, 1, 1)

            fake_images = generator(noise)

            outputs, activation_fake = discriminator(fake_images)
            _, activation_real = discriminator(right_images)

            activation_fake = torch.mean(activation_fake, 0)
            activation_real = torch.mean(activation_real, 0)

            g_loss = criterion(outputs, real_labels) \
                     + l2_coef * l2_loss(activation_fake, activation_real.detach()) \
                     + l1_coef * l1_loss(fake_images, right_images)

            g_loss.backward()
            optimG.step()
            
            if iteration % 5 == 0:
                logger.log_iteration_gan(epoch, d_loss, g_loss, real_score, fake_score)
                logger.draw(right_images, fake_images)

        logger.plot_epoch_w_scores(epoch)
        
        if epoch % 10 == 0:
            Utils.save_checkpoint(discriminator, generator, save_path, checkpoints_path, epoch)



In [97]:
train_vanilla_gan()

num epochs 50 num iterations 460 batch_size 64
Epoch: 0, d_loss= 0.716136, g_loss= 27.900574, D(X)= 0.765041, D(G(X))= 0.201918
Epoch: 0, d_loss= 0.961814, g_loss= 28.516281, D(X)= 0.823256, D(G(X))= 0.407148
Epoch: 0, d_loss= 0.725963, g_loss= 26.925980, D(X)= 0.745667, D(G(X))= 0.231526
Epoch: 0, d_loss= 1.009944, g_loss= 28.766560, D(X)= 0.878018, D(G(X))= 0.437449
Epoch: 0, d_loss= 1.040060, g_loss= 28.621420, D(X)= 0.805165, D(G(X))= 0.427112
Epoch: 0, d_loss= 0.821993, g_loss= 27.702543, D(X)= 0.711028, D(G(X))= 0.266303
Epoch: 0, d_loss= 0.747842, g_loss= 29.208015, D(X)= 0.848020, D(G(X))= 0.294920
Epoch: 0, d_loss= 1.003418, g_loss= 27.683783, D(X)= 0.718788, D(G(X))= 0.386601
Epoch: 0, d_loss= 2.128305, g_loss= 26.286978, D(X)= 0.142093, D(G(X))= 0.010929
Epoch: 0, d_loss= 1.070049, g_loss= 28.802185, D(X)= 0.419703, D(G(X))= 0.084820
Epoch: 0, d_loss= 0.832221, g_loss= 27.262581, D(X)= 0.752395, D(G(X))= 0.303289
Epoch: 0, d_loss= 1.167948, g_loss= 27.729086, D(X)= 0.348005,

Epoch: 1, d_loss= 0.787862, g_loss= 26.918907, D(X)= 0.577674, D(G(X))= 0.141908
Epoch: 1, d_loss= 0.806163, g_loss= 28.663485, D(X)= 0.522327, D(G(X))= 0.068923
Epoch: 1, d_loss= 0.503100, g_loss= 29.776794, D(X)= 0.812918, D(G(X))= 0.114532
Epoch: 1, d_loss= 0.692642, g_loss= 28.196888, D(X)= 0.802195, D(G(X))= 0.246151
Epoch: 1, d_loss= 0.718586, g_loss= 28.077129, D(X)= 0.634522, D(G(X))= 0.148564
Epoch: 1, d_loss= 0.968579, g_loss= 29.959993, D(X)= 0.869368, D(G(X))= 0.408771
Epoch: 1, d_loss= 0.742827, g_loss= 30.734606, D(X)= 0.778057, D(G(X))= 0.264314
Epoch: 1, d_loss= 1.422046, g_loss= 32.412834, D(X)= 0.951830, D(G(X))= 0.595585
Epoch: 1, d_loss= 0.693849, g_loss= 29.125397, D(X)= 0.705115, D(G(X))= 0.177037
Epoch: 1, d_loss= 0.647064, g_loss= 29.890354, D(X)= 0.633537, D(G(X))= 0.079987
Epoch: 1, d_loss= 0.765405, g_loss= 29.266857, D(X)= 0.796857, D(G(X))= 0.279808
Epoch: 1, d_loss= 0.543681, g_loss= 29.639416, D(X)= 0.758569, D(G(X))= 0.105927
Epoch: 1, d_loss= 0.762250, 

Epoch: 2, d_loss= 0.624866, g_loss= 28.831593, D(X)= 0.644454, D(G(X))= 0.076282
Epoch: 2, d_loss= 0.927556, g_loss= 31.862104, D(X)= 0.899784, D(G(X))= 0.403793
Epoch: 2, d_loss= 0.841343, g_loss= 30.559958, D(X)= 0.946718, D(G(X))= 0.345857
Epoch: 2, d_loss= 0.479017, g_loss= 29.614614, D(X)= 0.785424, D(G(X))= 0.070973
Epoch: 2, d_loss= 0.592443, g_loss= 29.540482, D(X)= 0.774622, D(G(X))= 0.145300
Epoch: 2, d_loss= 0.961117, g_loss= 27.875710, D(X)= 0.419493, D(G(X))= 0.063637
Epoch: 2, d_loss= 0.950864, g_loss= 28.303883, D(X)= 0.787584, D(G(X))= 0.382953
Epoch: 2, d_loss= 1.292197, g_loss= 29.922558, D(X)= 0.922801, D(G(X))= 0.526487
Epoch: 2, d_loss= 1.030292, g_loss= 26.770205, D(X)= 0.418956, D(G(X))= 0.079822
Epoch: 2, d_loss= 0.613762, g_loss= 27.431866, D(X)= 0.671001, D(G(X))= 0.097788
Epoch: 2, d_loss= 0.672234, g_loss= 28.213421, D(X)= 0.739237, D(G(X))= 0.192144
Epoch: 2, d_loss= 0.722841, g_loss= 28.915812, D(X)= 0.600952, D(G(X))= 0.118493
Epoch: 2, d_loss= 0.707721, 

Epoch: 3, d_loss= 0.728521, g_loss= 31.593122, D(X)= 0.866706, D(G(X))= 0.292531
Epoch: 3, d_loss= 0.900603, g_loss= 32.913277, D(X)= 0.927745, D(G(X))= 0.386973
Epoch: 3, d_loss= 0.732443, g_loss= 30.581793, D(X)= 0.951910, D(G(X))= 0.276023
Epoch: 3, d_loss= 0.580520, g_loss= 31.262947, D(X)= 0.906616, D(G(X))= 0.194699
Epoch: 3, d_loss= 0.709052, g_loss= 28.816139, D(X)= 0.595118, D(G(X))= 0.091096
Epoch: 3, d_loss= 0.893977, g_loss= 29.541698, D(X)= 0.922616, D(G(X))= 0.382999
Epoch: 3, d_loss= 1.427737, g_loss= 27.916920, D(X)= 0.270447, D(G(X))= 0.009030
Epoch: 3, d_loss= 0.661767, g_loss= 29.368698, D(X)= 0.863261, D(G(X))= 0.231194
Epoch: 3, d_loss= 0.608081, g_loss= 29.136097, D(X)= 0.865899, D(G(X))= 0.200686
Epoch: 3, d_loss= 0.891024, g_loss= 27.718693, D(X)= 0.462772, D(G(X))= 0.059111
Epoch: 3, d_loss= 0.498006, g_loss= 30.733974, D(X)= 0.764883, D(G(X))= 0.070044
Epoch: 3, d_loss= 0.608400, g_loss= 31.015806, D(X)= 0.889799, D(G(X))= 0.216588
Epoch: 3, d_loss= 0.451583, 

Epoch: 4, d_loss= 0.594868, g_loss= 28.445488, D(X)= 0.728491, D(G(X))= 0.126262
Epoch: 4, d_loss= 0.665035, g_loss= 28.368277, D(X)= 0.618584, D(G(X))= 0.066565
Epoch: 4, d_loss= 0.677159, g_loss= 28.476486, D(X)= 0.635056, D(G(X))= 0.092207
Epoch: 4, d_loss= 0.816303, g_loss= 28.080770, D(X)= 0.515652, D(G(X))= 0.067179
Epoch: 4, d_loss= 0.485301, g_loss= 29.874249, D(X)= 0.856302, D(G(X))= 0.102776
Epoch: 4, d_loss= 1.781177, g_loss= 27.126768, D(X)= 0.198103, D(G(X))= 0.006359
Epoch: 4, d_loss= 0.871266, g_loss= 30.310749, D(X)= 0.896771, D(G(X))= 0.348729
Epoch: 4, d_loss= 0.877407, g_loss= 28.688627, D(X)= 0.683546, D(G(X))= 0.289543
Epoch: 4, d_loss= 0.709363, g_loss= 29.586246, D(X)= 0.575143, D(G(X))= 0.043211
Epoch: 4, d_loss= 0.490589, g_loss= 31.281143, D(X)= 0.820525, D(G(X))= 0.093850
Epoch: 4, d_loss= 0.582978, g_loss= 30.118595, D(X)= 0.844650, D(G(X))= 0.180193
Epoch: 4, d_loss= 0.597156, g_loss= 30.202040, D(X)= 0.720640, D(G(X))= 0.115484
Epoch: 4, d_loss= 0.994760, 

...

Epoch: 50, d_loss= 0.397670, g_loss= 32.602741, D(X)= 0.933292, D(G(X))= 0.026806
Epoch: 50, d_loss= 0.373769, g_loss= 31.238804, D(X)= 0.836132, D(G(X))= 0.007714
Epoch: 50, d_loss= 0.384865, g_loss= 32.341743, D(X)= 0.950349, D(G(X))= 0.013830
Epoch: 50, d_loss= 0.413713, g_loss= 30.905739, D(X)= 0.946837, D(G(X))= 0.039699
Epoch: 50, d_loss= 0.367093, g_loss= 31.848909, D(X)= 0.852138, D(G(X))= 0.007936
Epoch: 50, d_loss= 0.450188, g_loss= 32.166115, D(X)= 0.973793, D(G(X))= 0.034782
Epoch: 50, d_loss= 0.358641, g_loss= 30.935263, D(X)= 0.867130, D(G(X))= 0.009425
Epoch: 50, d_loss= 0.366439, g_loss= 32.074257, D(X)= 0.891603, D(G(X))= 0.020523
Epoch: 50, d_loss= 0.406667, g_loss= 30.583809, D(X)= 0.779075, D(G(X))= 0.005049
Epoch: 50, d_loss= 0.357735, g_loss= 32.097832, D(X)= 0.867471, D(G(X))= 0.010288
Epoch: 50, d_loss= 0.361351, g_loss= 33.202660, D(X)= 0.907013, D(G(X))= 0.011560
Epoch: 50, d_loss= 0.367715, g_loss= 32.737289, D(X)= 0.869085, D(G(X))= 0.013158
Epoch: 50, d_los

Epoch: 51, d_loss= 0.388593, g_loss= 31.978235, D(X)= 0.807099, D(G(X))= 0.009861
Epoch: 51, d_loss= 0.363815, g_loss= 31.630877, D(X)= 0.920244, D(G(X))= 0.014768
Epoch: 51, d_loss= 0.364603, g_loss= 32.354263, D(X)= 0.914282, D(G(X))= 0.014437
Epoch: 51, d_loss= 0.359825, g_loss= 31.637646, D(X)= 0.883003, D(G(X))= 0.009883
Epoch: 51, d_loss= 0.366911, g_loss= 31.704231, D(X)= 0.844049, D(G(X))= 0.007200
Epoch: 51, d_loss= 0.376806, g_loss= 31.065399, D(X)= 0.938625, D(G(X))= 0.020795
Epoch: 51, d_loss= 0.374850, g_loss= 30.823694, D(X)= 0.933376, D(G(X))= 0.019035
Epoch: 51, d_loss= 0.357895, g_loss= 30.472467, D(X)= 0.874125, D(G(X))= 0.010427
Epoch: 51, d_loss= 0.408888, g_loss= 31.065842, D(X)= 0.958593, D(G(X))= 0.034561
Epoch: 51, d_loss= 0.389989, g_loss= 30.909924, D(X)= 0.948545, D(G(X))= 0.029102
Epoch: 51, d_loss= 0.368019, g_loss= 31.885035, D(X)= 0.911716, D(G(X))= 0.018423
Epoch: 51, d_loss= 0.376688, g_loss= 31.117964, D(X)= 0.830780, D(G(X))= 0.008238
Epoch: 51, d_los

Epoch: 52, d_loss= 0.357565, g_loss= 31.977375, D(X)= 0.899063, D(G(X))= 0.014760
Epoch: 52, d_loss= 0.361114, g_loss= 30.824532, D(X)= 0.837990, D(G(X))= 0.005398
Epoch: 52, d_loss= 0.349893, g_loss= 30.981342, D(X)= 0.911948, D(G(X))= 0.011381
Epoch: 52, d_loss= 0.360905, g_loss= 31.397141, D(X)= 0.874554, D(G(X))= 0.012017
Epoch: 52, d_loss= 0.374354, g_loss= 32.386597, D(X)= 0.944631, D(G(X))= 0.013444
Epoch: 52, d_loss= 0.357741, g_loss= 32.921841, D(X)= 0.898823, D(G(X))= 0.008964
Epoch: 52, d_loss= 0.364995, g_loss= 31.537123, D(X)= 0.925644, D(G(X))= 0.014031
Epoch: 52, d_loss= 0.366461, g_loss= 33.113480, D(X)= 0.923204, D(G(X))= 0.015497
Epoch: 52, d_loss= 0.381828, g_loss= 32.100750, D(X)= 0.818003, D(G(X))= 0.005738
Epoch: 52, d_loss= 0.371655, g_loss= 31.359695, D(X)= 0.868942, D(G(X))= 0.007445
Epoch: 52, d_loss= 0.370723, g_loss= 32.120144, D(X)= 0.880323, D(G(X))= 0.009041
Epoch: 52, d_loss= 0.356824, g_loss= 32.306606, D(X)= 0.903946, D(G(X))= 0.012498
Epoch: 52, d_los

Epoch: 53, d_loss= 0.556143, g_loss= 29.778744, D(X)= 0.832214, D(G(X))= 0.114051
Epoch: 53, d_loss= 0.627032, g_loss= 29.494034, D(X)= 0.676330, D(G(X))= 0.045129
Epoch: 53, d_loss= 0.505996, g_loss= 30.777025, D(X)= 0.903411, D(G(X))= 0.115614
Epoch: 53, d_loss= 0.467064, g_loss= 30.231516, D(X)= 0.849799, D(G(X))= 0.062568
Epoch: 53, d_loss= 0.448251, g_loss= 30.238838, D(X)= 0.862880, D(G(X))= 0.047599
Epoch: 53, d_loss= 0.428166, g_loss= 30.319550, D(X)= 0.861251, D(G(X))= 0.038426
Epoch: 53, d_loss= 0.546710, g_loss= 29.299522, D(X)= 0.720029, D(G(X))= 0.039944
Epoch: 53, d_loss= 0.473191, g_loss= 30.745148, D(X)= 0.879362, D(G(X))= 0.084371
Epoch: 53, d_loss= 0.447010, g_loss= 29.246815, D(X)= 0.807433, D(G(X))= 0.026540
Epoch: 53, d_loss= 0.437848, g_loss= 30.835522, D(X)= 0.795794, D(G(X))= 0.026617
Epoch: 53, d_loss= 0.444278, g_loss= 31.053501, D(X)= 0.786599, D(G(X))= 0.022102
Epoch: 53, d_loss= 0.410546, g_loss= 31.127987, D(X)= 0.883296, D(G(X))= 0.033209
Epoch: 53, d_los

KeyboardInterrupt: 

In [65]:
def predict_vanilla_gan(ds='flowers'):
    
    if ds == 'birds':
        dataset = Text2ImageDataset(config['birds_dataset_path'], split=2)
    if ds == 'flowers':
        dataset = Text2ImageDataset(config['flowers_dataset_path'], split=2)

    data_loader = DataLoader(dataset,
                            batch_size=batch_size,
                            shuffle=True)
    
    for sample in data_loader:
        right_images = sample['right_images']

        if not os.path.exists('results/{0}'.format(save_path)):
            os.makedirs('results/{0}'.format(save_path))

        right_images = Variable(right_images.float()).cuda()

        # Trained generator
        noise = Variable(torch.randn(right_images.size(0), 100)).cuda()
        noise = noise.view(noise.size(0), 100, 1, 1)
        fake_images = generator(noise)

        logger.draw(right_images, fake_images)

        for i, image in enumerate(fake_images):
            im = Image.fromarray(image.data.mul_(127.5).add_(127.5).byte().permute(1, 2, 0).cpu().numpy())
            im.save('results/gan/generated_images_90_test/{}.jpg'.format(i))
        break

In [66]:
predict_vanilla_gan()

## Train a GAN CLS

In [14]:
noise_dim = 100
batch_size = 64
num_workers = 1
lr = 0.0002
beta1 = 0.5
num_epochs = 90
l1_coef = 50
l2_coef = 200

vis_screen = 'gan_cls_loss'
checkpoints_path = 'checkpoints_gan_cls'
save_path = 'results'
logger = Logger(vis_screen)

Setting up a new session...


In [15]:
from models import gan_cls

generator = gan_cls.generator().cuda()
discriminator = gan_cls.discriminator().cuda()

pre_trained_disc = 'results/gan_cls/checkpoints_gan_cls/disc_90.pth'
pre_trained_gen = 'results/gan_cls/checkpoints_gan_cls/gen_90.pth'

if pre_trained_disc:
    discriminator.load_state_dict(torch.load(pre_trained_disc))
else:
    discriminator.apply(Utils.weights_init)

if pre_trained_gen:
    generator.load_state_dict(torch.load(pre_trained_gen))
else:
    generator.apply(Utils.weights_init)

optimD = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))
optimG = torch.optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))


In [16]:
def get_loss(ds='flowers', cls=True):
    criterion = nn.BCELoss()
    l2_loss = nn.MSELoss()
    l1_loss = nn.L1Loss()
    iteration = 0
        
    if ds == 'birds':
        dataset = Text2ImageDataset(config['birds_dataset_path'], split=0)
    if ds == 'flowers':
        dataset = Text2ImageDataset(config['flowers_dataset_path'], split=0)

    data_loader = DataLoader(dataset,
                            batch_size=batch_size,
                            shuffle=True)

    iteration = 0
    print("num iterations", len(data_loader), 'batch_size', batch_size)
    
    for epoch in range(0, num_epochs+1, 10):

        ### load the model #####
        pre_trained_disc = 'results/gan_cls/checkpoints/disc_{}.pth'.format(epoch)
        pre_trained_gen =  'results/gan_cls/checkpoints/gen_{}.pth'.format(epoch)

        generator = gan_cls.generator().cuda()
        discriminator = gan_cls.discriminator().cuda()

        generator.load_state_dict(torch.load(pre_trained_gen))
        discriminator.load_state_dict(torch.load(pre_trained_disc))

        ########################
        
        for sample in data_loader:
            iteration += 1
            right_images = sample['right_images']
            right_embed = sample['right_embed']
            wrong_images = sample['wrong_images']

            right_images = Variable(right_images.float()).cuda()
            right_embed = Variable(right_embed.float()).cuda()
            wrong_images = Variable(wrong_images.float()).cuda()

            real_labels = torch.ones(right_images.size(0))
            fake_labels = torch.zeros(right_images.size(0))

            smoothed_real_labels = torch.FloatTensor(Utils.smooth_label(real_labels.numpy(), -0.1))

            real_labels = Variable(real_labels).cuda()
            smoothed_real_labels = Variable(smoothed_real_labels).cuda()
            fake_labels = Variable(fake_labels).cuda()

            outputs, activation_real = discriminator(right_images, right_embed)
            real_loss = criterion(outputs, smoothed_real_labels)
            real_score = outputs

            if cls:
                outputs, _ = discriminator(wrong_images, right_embed)
                wrong_loss = criterion(outputs, fake_labels)
                wrong_score = outputs

            noise = Variable(torch.randn(right_images.size(0), 100)).cuda()
            noise = noise.view(noise.size(0), 100, 1, 1)
            fake_images = generator(right_embed, noise)
            outputs, _ = discriminator(fake_images, right_embed)
            fake_loss = criterion(outputs, fake_labels)
            fake_score = outputs

            d_loss = real_loss + fake_loss

            if cls:
                d_loss = d_loss + wrong_loss

            noise = Variable(torch.randn(right_images.size(0), 100)).cuda()
            noise = noise.view(noise.size(0), 100, 1, 1)
            fake_images = generator(right_embed, noise)
            outputs, activation_fake = discriminator(fake_images, right_embed)
            _, activation_real = discriminator(right_images, right_embed)

            activation_fake = torch.mean(activation_fake, 0)
            activation_real = torch.mean(activation_real, 0)

            g_loss = criterion(outputs, real_labels) \
                     + l2_coef * l2_loss(activation_fake, activation_real.detach()) \
                     + l1_coef * l1_loss(fake_images, right_images)

            if iteration % 5 == 0:
                logger.log_iteration_gan(epoch,d_loss, g_loss, real_score, fake_score)

        logger.plot_epoch_w_scores(epoch)

In [ ]:
def train_gan_cls(ds='flowers', cls=True, pre_trained_disc=None, pre_trained_gen=None):
    criterion = nn.BCELoss()
    l2_loss = nn.MSELoss()
    l1_loss = nn.L1Loss()
    iteration = 0
    
    if ds == 'birds':
        dataset = Text2ImageDataset(config['birds_dataset_path'], split=0)
    if ds == 'flowers':
        dataset = Text2ImageDataset(config['flowers_dataset_path'], split=0)

    data_loader = DataLoader(dataset,
                            batch_size=batch_size,
                            shuffle=True)

    iteration = 0
    
    print("num iterations", len(data_loader), 'batch_size', batch_size)

    for epoch in range(num_epochs):
        for sample in data_loader:
            iteration += 1
            right_images = sample['right_images']
            right_embed = sample['right_embed']
            wrong_images = sample['wrong_images']

            right_images = Variable(right_images.float()).cuda()
            right_embed = Variable(right_embed.float()).cuda()
            wrong_images = Variable(wrong_images.float()).cuda()

            real_labels = torch.ones(right_images.size(0))
            fake_labels = torch.zeros(right_images.size(0))

            # ======== One sided label smoothing ==========
            # Helps preventing the discriminator from overpowering the
            # generator adding penalty when the discriminator is too confident
            # =============================================
            smoothed_real_labels = torch.FloatTensor(Utils.smooth_label(real_labels.numpy(), -0.1))

            real_labels = Variable(real_labels).cuda()
            smoothed_real_labels = Variable(smoothed_real_labels).cuda()
            fake_labels = Variable(fake_labels).cuda()

            # Train the discriminator
            discriminator.zero_grad()
            outputs, activation_real = discriminator(right_images, right_embed)
            real_loss = criterion(outputs, smoothed_real_labels)
            real_score = outputs

            if cls:
                outputs, _ = discriminator(wrong_images, right_embed)
                wrong_loss = criterion(outputs, fake_labels)
                wrong_score = outputs

            noise = Variable(torch.randn(right_images.size(0), 100)).cuda()
            noise = noise.view(noise.size(0), 100, 1, 1)
            fake_images = generator(right_embed, noise)
            outputs, _ = discriminator(fake_images, right_embed)
            fake_loss = criterion(outputs, fake_labels)
            fake_score = outputs

            d_loss = real_loss + fake_loss

            if cls:
                d_loss = d_loss + wrong_loss

            d_loss.backward()
            optimD.step()

            # Train the generator
            generator.zero_grad()
            noise = Variable(torch.randn(right_images.size(0), 100)).cuda()
            noise = noise.view(noise.size(0), 100, 1, 1)
            fake_images = generator(right_embed, noise)
            outputs, activation_fake = discriminator(fake_images, right_embed)
            _, activation_real = discriminator(right_images, right_embed)

            activation_fake = torch.mean(activation_fake, 0)
            activation_real = torch.mean(activation_real, 0)


            #======= Generator Loss function============
            # This is a customized loss function, the first term is the regular cross entropy loss
            # The second term is feature matching loss, this measure the distance between the real and generated
            # images statistics by comparing intermediate layers activations
            # The third term is L1 distance between the generated and real images, this is helpful for the conditional case
            # because it links the embedding feature vector directly to certain pixel values.
            #===========================================
            g_loss = criterion(outputs, real_labels) \
                     + l2_coef * l2_loss(activation_fake, activation_real.detach()) \
                     + l1_coef * l1_loss(fake_images, right_images)

            g_loss.backward()
            optimG.step()

            if iteration % 5 == 0:
                logger.log_iteration_gan(epoch,d_loss, g_loss, real_score, fake_score)
                logger.draw(right_images, fake_images)

        logger.plot_epoch_w_scores(epoch)

        if (epoch) % 10 == 0:
            Utils.save_checkpoint(discriminator, generator, checkpoints_path, save_path, epoch)

In [ ]:
train_gan_cls()

In [77]:
def predict_gan_cls(ds='flowers'):
    
    if ds == 'birds':
        dataset = Text2ImageDataset(config['birds_dataset_path'], split=2)
    if ds == 'flowers':
        dataset = Text2ImageDataset(config['flowers_dataset_path'], split=2)

    data_loader = DataLoader(dataset,
                            batch_size=batch_size,
                            shuffle=False)
    
    for sample in data_loader:
        right_images = sample['right_images']
        right_embed = sample['right_embed']
        txt = sample['txt']

        if not os.path.exists('results/{0}'.format(save_path)):
            os.makedirs('results/{0}'.format(save_path))

        right_images = Variable(right_images.float()).cuda()
        right_embed = Variable(right_embed.float()).cuda()

        # Train the generator
        noise = Variable(torch.randn(right_images.size(0), 100)).cuda()
        noise = noise.view(noise.size(0), 100, 1, 1)
        fake_images = generator(right_embed, noise)

        logger.draw(right_images, fake_images)

        for image, t in zip(fake_images, txt):
            im = Image.fromarray(image.data.mul_(127.5).add_(127.5).byte().permute(1, 2, 0).cpu().numpy())
            t = t.strip()
            t = t.replace(".", '')
            im.save('results/gan_cls/generated_images_test/{}.jpg'.format(t.replace(" ", "_")[:100]))
            print(t)
        break

In [78]:
predict_gan_cls()

this flower has white petals as well as a purple pistil
this flower is white in color, and has a single large rounded petal
this flower has many pointy pale red petals that overlap and a broad, large group of white stamen in the center
this flower has a thick green pedicel and rounded yellow petals
this flower is yellow and black in color, with petals that are spotted near the center
this flower has smooth white petals with orange stamen in the center of it
the flower is so vivid with purple color and has petals that are soft and stamen are white and sticking out from the centre
this flower has petals that are yellow and closed up
the flower has a thin green pedicel with star shaped purple petals and white anther
this flower is orange, blue, and green in color, and has petals that are pointed
the sepals are green and the flower is blue in color
this flower is white and purple in color, and has petals that are wavy and thin
the petals of this flower are pink and the pistil is red
this f